#**Import Modules**

Selenium, Openpyxl, Files, Time

In [0]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2,
         "download_restrictions": 3}
chrome_options.add_experimental_option("prefs", prefs)
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Fetched 88.7 kB in 2s (45.7 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree     

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [0]:
from selenium.common.exceptions import *

In [0]:
import time

In [0]:
import openpyxl

In [0]:
from google.colab import files

#**Fetch all Volume Links**

In [0]:
MAINURL = "http://www.kluwerlawonline.com/toc.php?pubcode=TRAD"
volUrls = []

# access main page
wd.get(MAINURL)
time.sleep(0.5)

a_tags = wd.find_elements_by_xpath("//a[@href]")
for tag in a_tags:
    if "Volume" in tag.get_attribute("href"):
        volUrls.append(tag.get_attribute("href"))

In [0]:
# check missing volume no.
tmps = []

for volurl in volUrls:
    tmp = volurl[volurl.index("Volume") + 7:]
    tmp = tmp[:tmp.index("+")]
    if tmp not in tmps:
        tmps.append(tmp)

for i in range(54):
    if str(i + 1) not in tmps:
        print(i + 1)

#**Fetch and Write Data from each Volumes**

In [0]:
wb = openpyxl.Workbook()
ws = wb.active

In [0]:
 r = 2

 # write header
 ws.cell(row = 1, column = 1).value = 'journal'
 ws.cell(row = 1, column = 2).value = 'volume no.'
 ws.cell(row = 1, column = 3).value = 'title'
 ws.cell(row = 1, column = 4).value = 'keywords'
 ws.cell(row = 1, column = 5).value = 'authors'

 for volurl in volUrls:
    # access volume page
    wd.get(volurl)
    time.sleep(1)

    # fetch volume no. from url
    print("Reading Volume, " + volurl)
    vol = volurl[volurl.index("Volume") + 7:]
    vol = vol[:vol.index("+")]

    # fetch article urls
    articleUrls = []
    a_tags = wd.find_elements_by_xpath("//td[@class='to_document_hitlist']/a[@href]")
    for tag in a_tags:
        articleUrls.append(tag.get_attribute("href"))

    for articleurl in articleUrls:
        # access article page
        wd.get(articleurl)
        time.sleep(0.8)
        #print("Reading article, " + articleurl)

        # fetch title
        try:
            header = wd.find_element_by_id("citation").get_attribute("innerText")
            title = header[header.index("\'") + 1:]
            title = title[:title.index("\'")]
        except ValueError:
            continue
        except NoSuchElementException:
            continue
        #print(title)
        
        # fetch authors
        authors = []
        try:
            authors = header[header.index("\n") + 1:header.index("\'")].split(",")
            while ' ' in authors: 
                authors.remove(' ')
        except ValueError:
            continue
        if(len(authors) == 0):
            continue
        #print(authors)

        # fetch keywords
        keywords = []
        try:
            keywords = wd.find_element_by_xpath("//span[contains(text(), 'Keywords:')]").find_element_by_xpath("..").get_attribute("innerText")
            keywords = keywords[keywords.index("\n") + 1:].replace("; ", ", ").split(", ")
            while ' ' in keywords:
                keywords.remove(' ')
            for j in range(len(keywords)):
                if "JEL" in keywords[j]:
                    keywords[j] = keywords[j][:keywords[j].index("JEL")]
                    if keywords[j][len(keywords[j]) - 1] == ' ':
                        keywords[j] = keywords[j][:len(keywords[j]) - 1]
                    keywords = keywords[:j + 1]
                    break
        except ValueError:
            continue
        except NoSuchElementException:
            continue
        if(len(keywords) == 0):
            continue
        #print(keywords)

        # write data to excel file
        print("Writing, " + title)
        ws.cell(row = r, column = 1).value = "Journal of World Trade"
        ws.cell(row = r, column = 2).value = vol
        ws.cell(row = r, column = 3).value = title
        for j in range(len(keywords)):
            ws.cell(row = r + j, column = 4).value = keywords[j]
        for j in range(len(authors)):
            ws.cell(row = r + j, column = 5).value = authors[j]
        r += max([len(keywords), len(authors)])
    
    # backup for every ten volumes
    if(volUrls.index(volurl) % 20 == 0):
        print("Saving backup file")
        wb.save(filename = "journal_of_world_trade_backup.xlsx")
        time.sleep(20)
        #files.download("journal_of_world_trade_backup.xlsx")

Reading Volume, http://www.kluwerlawonline.com/toc.php?area=Journals&mode=bypub&level=5&values=Journals%7E%7EJournal+of+World+Trade%7EVolume+54+%282020%29
Writing, Is EU GSP+ Fostering Good Governance? Results from a New GSP+ Compliance Index
Writing, Regulating Trade in Forest-Risk Commodities
Writing, ‘E-commerce’ for India in a Developing World: An Int’l Trade Law Perspective
Writing, Developing Countries and Domestic Support Measures in Agriculture: Walking a Tightrope
Writing, A Crisis or an Opportunity? The Trade War Between the US and China in the Solar PV Sector
Writing, Asymmetrical Fairness: China’s Use of Antidumping Measures
Writing, Regulating the State Capitalism: Is There an Optimal Regulatory Model for Sovereign Wealth Fund Investment?
Saving backup file
Reading Volume, http://www.kluwerlawonline.com/toc.php?area=Journals&mode=bypub&level=5&values=Journals%7E%7EJournal+of+World+Trade%7EVolume+53+%282019%29
Reading Volume, http://www.kluwerlawonline.com/toc.php?area=Jour

#**Save File**

In [0]:
wb.save(filename = "journal_of_world_trade_.xlsx")

#**Download FIle**

In [0]:
files.download("journal_of_world_trade.xlsx")